# Projekt ZUM 25L - Temat 16
Prowadzący projektu - Paweł Cichosz

Autorzy:
- Igor Matynia
- Filip Gryz

# Temat projektu
Nienadzorowana detekcja anomalii na podstawie niepodobieństwa do grup wyznaczanych za
pomocą algorytmów grupowania. Implementacja w formie opakowania umożliwiającego użycie
różnych algorytmów grupowania dostępnych w środowisku R lub Python i różnych miar niepodobieństwa. Porównanie z nienadzorowaną detekcją anomalii za pomocą algorytmów klasyfikacji jednoklasowej dostępnych w środowisku R lub Python.

# Interpretacja tematu
...

# Założenia projektowe
...

## Zmiany względem założeń wstepnych
...



# Instalacja wymaganych pakietów

In [ ]:
!pip install -q scikit-learn matplotlib numpy pandas

: 

# Implementacja autorskiego framework'u do detekcji anomalii
Opis implementacji

## Eksperymenty
...

## Wyniki i wnioski eksperymentów

...

# Implementacja estymatora ilości klastrów zbioru danych

In [ ]:
from sklearn.base import BaseEstimator, ClusterMixin
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
import numpy as np
import warnings

class ClusterNumberEstimator(BaseEstimator, ClusterMixin):
    """
    An unsupervised estimator to determine the optimal number of clusters
    based on internal validation metrics (Silhouette score and Davies-Bouldin index).

    This estimator iteratively fits KMeans for increasing k and stops when validation
    metrics no longer improve. The final number of clusters is available as `n_clusters_`.
    """
    def __init__(self, max_k=20, min_k=2, random_state=None, verbose=False):
        self.max_k = max_k
        self.min_k = min_k
        self.random_state = random_state
        self.verbose = verbose

    def fit(self, X, y=None):
        best_k = self.min_k
        best_score = -1
        best_dbi = np.inf
        prev_score = -np.inf
        prev_dbi = np.inf

        self._scores = []
        self._dbis = []

        for k in range(self.min_k, self.max_k + 1):
            kmeans = KMeans(n_clusters=k, random_state=self.random_state, n_init='auto')
            labels = kmeans.fit_predict(X)

            try:
                silhouette = silhouette_score(X, labels)
                dbi = davies_bouldin_score(X, labels)
            except Exception as e:
                warnings.warn(f"Could not compute validation metrics for k={k}: {e}")
                continue

            self._scores.append(silhouette)
            self._dbis.append(dbi)

            if self.verbose:
                print(f"k={k}, Silhouette={silhouette:.4f}, DBI={dbi:.4f}")

            if silhouette < prev_score or dbi > prev_dbi:
                break

            prev_score = silhouette
            prev_dbi = dbi
            best_k = k
            best_labels = labels
            best_centers = kmeans.cluster_centers_

        self.n_clusters_ = best_k
        self.labels_ = best_labels
        self.cluster_centers_ = best_centers
        return self

    def predict(self, X):
        if not hasattr(self, "cluster_centers_"):
            raise RuntimeError("Estimator has not been fitted yet.")
        return KMeans(n_clusters=self.n_clusters_, init=self.cluster_centers_, n_init=1).fit_predict(X)

    def get_validation_metrics(self):
        return self._scores, self._dbis


## Eksperymenty

## Wyniki i wnioski eksperymentów

# Podsumowanie
...